In [3]:
import pandas as pd
import tarfile
import json
import polars as pl

In [2]:
pip install polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 3.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
#Buisness

# Define the JSON file path
json_file_path = "yelp_academic_dataset_business.json"

# Read the JSON file in batches
buinsess = pl.scan_ndjson(json_file_path)

# Select only the 'user_id' and 'text' columns
buinsess_subset = buinsess.select(['business_id', 'name']).collect(streaming = True)

In [5]:
#Users

# Define the JSON file path
json_file_path = "yelp_academic_dataset_user.json"

# Read the JSON file in batches
users = pl.scan_ndjson(json_file_path)

# Select only the 'user_id' and 'text' columns
users_subset = users.select(['user_id', 'elite']).collect(streaming = True)

In [6]:
#Reviews

# Define the JSON file path
json_file_path = "yelp_academic_dataset_review.json"

# Read the JSON file in batches
reviews = pl.scan_ndjson(json_file_path)

# Select only the 'user_id' and 'text' columns
reviews_subset = reviews.select(['user_id', 'text', 'date', 'business_id', 'stars', 'useful', 'funny', 'cool']).collect(streaming = True)


In [84]:
stars_counts = reviews_subset['stars'].value_counts()

In [85]:
stars_counts

stars,counts
f64,u32
1.0,1069561
2.0,544240
3.0,691934
4.0,1452918
5.0,3231627


In [7]:
# Define the fraction of rows to sample
frac = 0.005  # For example, to sample 10% of the rows

# Define the column(s) to control the sample
control_columns = ["stars", "useful", "funny", "cool"]  # Replace with your desired columns

# Group by the control columns
grouped_df = reviews_subset.groupby(control_columns)

# Convert grouped DataFrame to a list of DataFrames
grouped_dfs = [group for group in grouped_df]

# Sample each DataFrame individually
sampled_dfs = [group.sample(fraction=frac) for group in grouped_dfs]

# Concatenate the sampled DataFrames back together
sampled_df = pl.concat(sampled_dfs, axis=0)

/var/folders/qz/c0nnbl790814q7f1jrtp6v1w0000gn/T/ipykernel_57109/3437540205.py:8: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  grouped_df = reviews_subset.groupby(control_columns)


KeyboardInterrupt: 

In [8]:
reviews_users_df = reviews_subset.join(users_subset, on='user_id', how='inner')

In [9]:
elite_users_df = reviews_users_df.filter(pl.col('elite') != "")

In [52]:
elite_users_df.head()

user_id,text,date,business_id,elite
str,str,str,str,str
"""8g_iMtfSiwikVn…","""Family diner. …","""2014-02-05 20:…","""YjUWPpI6HXG530…","""2014,2015,2016…"
"""9OAtfnWag-ajVx…","""Love going her…","""2014-06-27 22:…","""lj-E32x9_FA7Gm…","""2014,2015,2016…"
"""4Uh27DgGzsp6Pq…","""The bun makes …","""2011-10-27 17:…","""otQS34_MymijPT…","""2011,2012"""
"""j2wlzrntrbKwyO…","""The hubby and …","""2014-08-10 19:…","""rBdG_23USc7Dle…","""2011,2012,2013…"
"""IQsF3Rc6IgCzjV…","""My absolute fa…","""2014-11-12 15:…","""eFvzHawVJofxSn…","""2018"""


In [10]:
elite_users_df = elite_users_df.select(
    "*",
    pl.col("date").apply(lambda date: date.split("-")[0]).alias("comment_year")
)

/var/folders/qz/c0nnbl790814q7f1jrtp6v1w0000gn/T/ipykernel_57109/1015227033.py:3: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col("date").apply(lambda date: date.split("-")[0]).alias("comment_year")


In [56]:
filtered_data = elite_users_df.filter(
    pl.col("comment_year").is_in(pl.col("elite"))
)

In [60]:
filtered_data = filtered_data.filter(
    pl.col("comment_year").is_in(pl.col("elite").str.split(",").explode())
)

In [11]:
filtered_data = elite_users_df.filter(
    pl.col("elite").str.contains(pl.col("comment_year").cast(pl.Utf8()))
)

In [12]:
reviews_buisness = filtered_data.join(buinsess_subset, on='business_id', how='inner')

In [13]:
reviews_buisness = reviews_buisness.select(
    "*", 
    pl.col('text').str.lengths().alias('text_length')
)


/var/folders/qz/c0nnbl790814q7f1jrtp6v1w0000gn/T/ipykernel_57109/1454544245.py:3: DeprecationWarning: `lengths` is deprecated. It has been renamed to `len_bytes`.
  pl.col('text').str.lengths().alias('text_length')


In [15]:
reviews_buisness.shape

(1252228, 12)

In [42]:
# Define the fraction of rows to sample
frac = 0.01  # For example, to sample 0.5% of the rows

# Define the column(s) to control the sample
control_columns = ["stars", "useful", "funny", "cool"]  # Replace with your desired columns

# Group by the control columns
grouped_df = reviews_buisness.groupby(control_columns)

# Sample each group individually and concatenate them back together
sampled_df = pl.concat([
    group[1].sample(fraction=frac) for group in grouped_df
])

/var/folders/qz/c0nnbl790814q7f1jrtp6v1w0000gn/T/ipykernel_57109/3966149111.py:8: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  grouped_df = reviews_buisness.groupby(control_columns)


user_id,text,date,business_id,stars,useful,funny,cool,elite,comment_year,name,text_length
str,str,str,str,f64,i64,i64,i64,str,str,str,u32
"""DImCfo-rwcpkof…","""Super yummy f…","""2018-08-30 19:…","""tbnajfmD2VlxDQ…",4.0,0,0,0,"""2017,2018,2019…","""2018""","""Choolaah""",220
"""wlZDmZvRFXSSSu…","""Amazing new co…","""2013-06-25 12:…","""Qlmusk0BBY0kMO…",4.0,0,0,0,"""2013,2014""","""2013""","""Kingfish""",383
"""aIl6bcdpch8spq…","""This was a nic…","""2011-08-07 19:…","""CBygzH3_5D_W23…",4.0,0,0,0,"""2011""","""2011""","""Independence N…",474
"""wfi0T-xq_P5sKQ…","""This Second Cu…","""2013-08-05 15:…","""J3v9wfVtxcQ2DZ…",4.0,0,0,0,"""2011,2012,2013…","""2013""","""Second Cup Cof…",796
"""ecHwvfy3wOAih_…","""Better than th…","""2014-05-19 03:…","""_W68O1txBhPTng…",4.0,0,0,0,"""2014,2015,2016…","""2014""","""Village Inn""",326


In [43]:
yelp_verified = sampled_df.select(pl.col(["text","date", "name", "text_length"]))

In [47]:
yelp_verified = sampled_df.rename({"name": "business_name"})

In [48]:
yelp_verified.head()

user_id,text,date,business_id,stars,useful,funny,cool,elite,comment_year,business_name,text_length
str,str,str,str,f64,i64,i64,i64,str,str,str,u32
"""UlPCp6kFGGUSKy…","""PSA: CASH ONLY…","""2019-11-13 14:…","""d48Xrx8MhGtdaL…",5.0,0,0,0,"""2019,20,20,202…","""2019""","""Cafe Diem""",622
"""t2ZKf-CjGthLam…","""When you're cr…","""2019-11-09 19:…","""ngU4740twiB222…",5.0,0,0,0,"""2019,20,20,202…","""2019""","""Cocina Latina""",235
"""9fOezqmM4pYOHC…","""Loved our chee…","""2019-12-03 17:…","""8xTHtLoNIwdpf0…",5.0,0,0,0,"""2015,2016,2017…","""2019""","""By George! Piz…",261
"""UHuNFy-CDLj_SV…","""Such fun! Grea…","""2021-09-25 14:…","""29sZgoR7VN_3ck…",5.0,0,0,0,"""20,20,2021""","""2021""","""Izakaya Japane…",265
"""DbiZXAui0L2LGH…","""Really cool sp…","""2018-08-24 16:…","""JUlsvVAvZvGHWF…",5.0,0,0,0,"""2018,2019""","""2018""","""El Camino Real…",975


In [49]:
# Define the file path
path = "./yelp_verified_slim.csv"

# Write the DataFrame to a CSV file
yelp_verified.write_csv(path, separator=",")

In [50]:
yelp_verified.shape

(11106, 12)